In [1]:
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
# read csv
# https://research.unsw.edu.au/projects/unsw-nb15-dataset
# According to the website above csv is wrongly saved, so I change the name
# The number of records in the training set is 175,341 records and the testing set is 82,332 records from the different types, attack and normal.
df = pd.read_csv('../../Data/UNSW_NB15_testing-set.csv')

In [3]:
# fix seed
# Pytorch
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [4]:
# nominal to numeric of data
# proto                 object
# service               object
# state                 object

# proto to numeric
# proto_mapping = {'xxx':2, 'xxx':1, 'xxx':0}
# data['proto'] = data['proto'].map(proto_mapping)

# proto to numeric
proto_le = LabelEncoder()
df['proto'] = proto_le.fit_transform(df['proto'])
 
# service to numeric
service_le = LabelEncoder()
df['service'] = service_le.fit_transform(df['service'])

# state to numeric
state_le = LabelEncoder()
df['state'] = state_le.fit_transform(df['state'])

# nominal to numeric of data
# attack_cat            object

# target to numeric
df['attack_cat2']=df['attack_cat']
attack_cat2_le = LabelEncoder()
df['attack_cat2'] = attack_cat2_le.fit_transform(df['attack_cat2'])
df_processed = pd.get_dummies(df, columns=(['attack_cat']))


df_processed.head(10)

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,attack_cat_Analysis,attack_cat_Backdoor,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Reconnaissance,attack_cat_Shellcode,attack_cat_Worms
0,1,0.121478,113,0,2,6,4,258,172,74.087490,...,0,0,0,0,0,0,1,0,0,0
1,2,0.649902,113,0,2,14,38,734,42014,78.473372,...,0,0,0,0,0,0,1,0,0,0
2,3,1.623129,113,0,2,8,16,364,13186,14.170161,...,0,0,0,0,0,0,1,0,0,0
3,4,1.681642,113,3,2,12,12,628,770,13.677108,...,0,0,0,0,0,0,1,0,0,0
4,5,0.449454,113,0,2,10,6,534,268,33.373826,...,0,0,0,0,0,0,1,0,0,0
5,6,0.380537,113,0,2,10,6,534,268,39.417980,...,0,0,0,0,0,0,1,0,0,0
6,7,0.637109,113,0,2,10,8,534,354,26.683033,...,0,0,0,0,0,0,1,0,0,0
7,8,0.521584,113,0,2,10,8,534,354,32.593026,...,0,0,0,0,0,0,1,0,0,0
8,9,0.542905,113,0,2,10,8,534,354,31.313031,...,0,0,0,0,0,0,1,0,0,0
9,10,0.258687,113,0,2,10,6,534,268,57.985135,...,0,0,0,0,0,0,1,0,0,0


In [5]:
df_train, df_test = train_test_split(df_processed, test_size=0.3, random_state=0)

In [6]:
#df_train.dtypes
df_train.shape

(122738, 55)

In [7]:
df_train.head(10)

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,attack_cat_Analysis,attack_cat_Backdoor,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Reconnaissance,attack_cat_Shellcode,attack_cat_Worms
41479,41480,0.581798,113,0,2,10,6,588,268,2.578214e+01,...,0,0,0,0,0,0,1,0,0,0
174119,174120,0.000009,119,2,3,2,0,114,0,1.111111e+05,...,0,0,0,0,0,1,0,0,0,0
39585,39586,0.294033,113,4,2,8,12,424,8824,6.461860e+01,...,0,0,0,0,0,0,1,0,0,0
23269,23270,0.001059,119,2,0,2,2,146,178,2.832861e+03,...,0,0,0,0,0,0,1,0,0,0
35227,35228,1.433471,113,3,2,52,54,2934,3742,7.324878e+01,...,0,0,0,0,0,0,1,0,0,0
157804,157805,0.000003,119,2,3,2,0,114,0,3.333333e+05,...,0,0,0,0,0,1,0,0,0,0
170646,170647,1.434152,113,4,2,10,8,450,782,1.185370e+01,...,0,0,0,1,0,0,0,0,0,0
102714,102715,1.725872,113,5,2,10,10,798,1730,1.100893e+01,...,0,0,1,0,0,0,0,0,0,0
138365,138366,0.000001,119,2,3,2,0,114,0,1.000000e+06,...,0,0,0,0,0,1,0,0,0,0
7840,7841,0.528823,113,9,2,52,42,37372,3380,1.758622e+02,...,0,0,0,0,0,0,1,0,0,0


In [8]:
data_train = df_train.drop(['id', 'label', 'attack_cat2','attack_cat_Analysis','attack_cat_Backdoor','attack_cat_DoS','attack_cat_Exploits','attack_cat_Fuzzers','attack_cat_Generic','attack_cat_Normal','attack_cat_Reconnaissance','attack_cat_Shellcode','attack_cat_Worms'], axis=1)
analysis_train = df_train.iloc[:,-10]
backdoor_train = df_train.iloc[:,-9]
dos_train = df_train.iloc[:,-8]
exploits_train = df_train.iloc[:,-7]
fuzzers_train = df_train.iloc[:,-6]
generic_train = df_train.iloc[:,-5]
normal_train = df_train.iloc[:,-4]
reconnaissance_train = df_train.iloc[:,-3]
shellcode_train = df_train.iloc[:,-2]
worms_train = df_train.iloc[:,-1]

attack_train = df_train.iloc[:,-11]
data_train.head(10)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
41479,0.581798,113,0,2,10,6,588,268,2.578214e+01,254,...,2,1,1,1,0,0,0,1,5,0
174119,0.000009,119,2,3,2,0,114,0,1.111111e+05,254,...,15,15,15,31,0,0,0,15,31,0
39585,0.294033,113,4,2,8,12,424,8824,6.461860e+01,31,...,11,1,1,5,0,0,0,3,6,0
23269,0.001059,119,2,0,2,2,146,178,2.832861e+03,31,...,2,1,1,1,0,0,0,4,2,0
35227,1.433471,113,3,2,52,54,2934,3742,7.324878e+01,31,...,4,1,1,5,1,1,0,6,2,0
157804,0.000003,119,2,3,2,0,114,0,3.333333e+05,254,...,12,12,12,14,0,0,0,12,14,0
170646,1.434152,113,4,2,10,8,450,782,1.185370e+01,62,...,2,1,1,2,0,0,0,4,1,0
102714,1.725872,113,5,2,10,10,798,1730,1.100893e+01,62,...,1,1,1,1,0,0,1,1,1,0
138365,0.000001,119,2,3,2,0,114,0,1.000000e+06,254,...,16,16,16,18,0,0,0,16,18,0
7840,0.528823,113,9,2,52,42,37372,3380,1.758622e+02,31,...,2,1,1,3,0,0,0,4,1,0


In [9]:
#min-max scaling
data_train_norm = (data_train - data_train.min()) / (data_train.max() - data_train.min())
data_train_norm = data_train_norm.fillna(0)

In [10]:
data_train_norm.shape
data_train_norm.head(10)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
41479,9.696635e-03,0.856061,0.000000,0.285714,0.000936,0.000553,0.000042,0.000018,0.000026,0.996078,...,0.02,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.065574,0.0
174119,1.500000e-07,0.901515,0.166667,0.428571,0.000104,0.000000,0.000005,0.000000,0.111111,0.996078,...,0.28,0.28,0.311111,0.483871,0.00,0.00,0.000000,0.237288,0.491803,0.0
39585,4.900551e-03,0.856061,0.333333,0.285714,0.000728,0.001106,0.000029,0.000609,0.000065,0.121569,...,0.20,0.00,0.000000,0.064516,0.00,0.00,0.000000,0.033898,0.081967,0.0
23269,1.765000e-05,0.901515,0.166667,0.000000,0.000104,0.000184,0.000008,0.000012,0.002833,0.121569,...,0.02,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.050847,0.016393,0.0
35227,2.389119e-02,0.856061,0.250000,0.285714,0.005304,0.004977,0.000223,0.000258,0.000073,0.121569,...,0.06,0.00,0.000000,0.064516,0.25,0.25,0.000000,0.084746,0.016393,0.0
157804,5.000001e-08,0.901515,0.166667,0.428571,0.000104,0.000000,0.000005,0.000000,0.333333,0.996078,...,0.22,0.22,0.244444,0.209677,0.00,0.00,0.000000,0.186441,0.213115,0.0
170646,2.390254e-02,0.856061,0.333333,0.285714,0.000936,0.000737,0.000031,0.000054,0.000012,0.243137,...,0.02,0.00,0.000000,0.016129,0.00,0.00,0.000000,0.050847,0.000000,0.0
102714,2.876454e-02,0.856061,0.416667,0.285714,0.000936,0.000922,0.000058,0.000119,0.000011,0.243137,...,0.00,0.00,0.000000,0.000000,0.00,0.00,0.033333,0.000000,0.000000,0.0
138365,1.666667e-08,0.901515,0.166667,0.428571,0.000104,0.000000,0.000005,0.000000,1.000000,0.996078,...,0.30,0.30,0.333333,0.274194,0.00,0.00,0.000000,0.254237,0.278689,0.0
7840,8.813718e-03,0.856061,0.750000,0.285714,0.005304,0.003871,0.002879,0.000233,0.000176,0.121569,...,0.02,0.00,0.000000,0.032258,0.00,0.00,0.000000,0.050847,0.000000,0.0


In [11]:
# preparing 10 types of binary classification set
train_X = torch.tensor(data_train_norm.values, dtype=torch.float32)

train_analysis_Y = torch.tensor(analysis_train.values, dtype=torch.long) 
train_backdoor_Y = torch.tensor(backdoor_train.values, dtype=torch.long) 
train_dos_Y = torch.tensor(dos_train.values, dtype=torch.long) 
train_exploits_Y = torch.tensor(exploits_train.values, dtype=torch.long) 
train_fuzzers_Y = torch.tensor(fuzzers_train.values, dtype=torch.long) 
train_generic_Y = torch.tensor(generic_train.values, dtype=torch.long) 
train_normal_Y = torch.tensor(normal_train.values, dtype=torch.long) 
train_reconnaissance_Y = torch.tensor(reconnaissance_train.values, dtype=torch.long) 
train_shellcode_Y = torch.tensor(shellcode_train.values, dtype=torch.long) 
train_worms_Y = torch.tensor(worms_train.values, dtype=torch.long) 

train_analysis = TensorDataset(train_X, train_analysis_Y)
train_backdoor = TensorDataset(train_X, train_backdoor_Y)
train_dos = TensorDataset(train_X, train_dos_Y)
train_exploits = TensorDataset(train_X, train_exploits_Y)
train_fuzzers = TensorDataset(train_X, train_fuzzers_Y)
train_generic = TensorDataset(train_X, train_generic_Y)
train_normal = TensorDataset(train_X, train_normal_Y)
train_reconnaissance = TensorDataset(train_X, train_reconnaissance_Y)
train_shellcode = TensorDataset(train_X, train_shellcode_Y)
train_worms = TensorDataset(train_X, train_worms_Y)


In [12]:
train_analysis_loader = DataLoader(train_analysis, batch_size=100, shuffle=True)
train_backdoor_loader = DataLoader(train_backdoor, batch_size=100, shuffle=True)
train_dos_loader = DataLoader(train_dos, batch_size=100, shuffle=True)
train_exploits_loader = DataLoader(train_exploits, batch_size=100, shuffle=True)
train_fuzzers_loader = DataLoader(train_fuzzers, batch_size=100, shuffle=True)
train_generic_loader = DataLoader(train_generic, batch_size=100, shuffle=True)
train_normal_loader = DataLoader(train_normal, batch_size=100, shuffle=True)
train_reconnaissance_loader = DataLoader(train_reconnaissance, batch_size=100, shuffle=True)
train_shellcode_loader = DataLoader(train_shellcode, batch_size=100, shuffle=True)
train_worms_loader = DataLoader(train_worms, batch_size=100, shuffle=True)
# drop_last = True

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 2)
        self.bc1 = nn.BatchNorm1d(100)
        self.bc2 = nn.BatchNorm1d(100)
        self.bc3 = nn.BatchNorm1d(100)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bc1(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc2(x)
        x = self.bc2(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc3(x)
        x = self.bc3(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc4(x)
        # return F.log_softmax(x, dim=1)
        return x

model_analysis = Net()
model_backdoor = Net()
model_dos = Net()
model_exploits = Net()
model_fuzzers = Net()
model_generic = Net()
model_normal = Net()
model_reconnaissance = Net()
model_shellcode = Net()
model_worms = Net()

In [14]:
print(torch.cuda.is_available())

True


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_analysis = model_analysis.to(device)
model_backdoor = model_backdoor.to(device)
model_dos = model_dos.to(device)
model_exploits = model_exploits.to(device)
model_fuzzers = model_fuzzers.to(device)
model_generic = model_generic.to(device)
model_normal = model_normal.to(device)
model_reconnaissance = model_reconnaissance.to(device)
model_shellcode = model_shellcode.to(device)
model_worms = model_worms.to(device)
print(device)

cuda


In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.03)
optimizer_analysis = torch.optim.Adam(model_analysis.parameters(), lr=0.03)
optimizer_backdoor = torch.optim.Adam(model_backdoor.parameters(), lr=0.03)
optimizer_dos = torch.optim.Adam(model_dos.parameters(), lr=0.03)
optimizer_exploits = torch.optim.Adam(model_exploits.parameters(), lr=0.03)
optimizer_fuzzers = torch.optim.Adam(model_fuzzers.parameters(), lr=0.03)
optimizer_generic = torch.optim.Adam(model_generic.parameters(), lr=0.03)
optimizer_normal = torch.optim.Adam(model_normal.parameters(), lr=0.03)
optimizer_reconnaissance = torch.optim.Adam(model_reconnaissance.parameters(), lr=0.03)
optimizer_shellcode = torch.optim.Adam(model_shellcode.parameters(), lr=0.03)
optimizer_worms = torch.optim.Adam(model_worms.parameters(), lr=0.03)

In [18]:
#Training the machine learning model
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_analysis.train()
    loss_list=[]
    for train_x, train_y in train_analysis_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_analysis.zero_grad()
        output = model_analysis(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_analysis.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_analysis_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

In [19]:
with torch.no_grad():
    correct = 0
    model_analysis.eval()
    train_x, train_y = Variable(train_X), Variable(train_analysis_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_analysis = model_analysis(train_x)
    pred = torch.max(output_analysis.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 121461/122738 (99%)



In [20]:
#output.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()

In [21]:
twoS_train_data = pd.DataFrame()
twoS_train_data["analysis_pred"]=output_analysis.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

,analysis_pred
0,1.124398e-09
1,1.652119e-13
2,0.000000e+00
3,2.162503e-31
4,0.000000e+00
5,3.420381e-14
6,7.428961e-08
7,4.485195e-02
8,2.902930e-28
9,0.000000e+00


In [22]:
testdata = df_test.drop(['id', 'label', 'attack_cat2','attack_cat_Analysis','attack_cat_Backdoor','attack_cat_DoS','attack_cat_Exploits','attack_cat_Fuzzers','attack_cat_Generic','attack_cat_Normal','attack_cat_Reconnaissance','attack_cat_Shellcode','attack_cat_Worms'], axis=1)

analysis_test = df_test.iloc[:,-10]
backdoor_test = df_test.iloc[:,-9]
dos_test = df_test.iloc[:,-8]
exploits_test = df_test.iloc[:,-7]
fuzzers_test = df_test.iloc[:,-6]
generic_test = df_test.iloc[:,-5]
normal_test = df_test.iloc[:,-4]
reconnaissance_test = df_test.iloc[:,-3]
shellcode_test = df_test.iloc[:,-2]
worms_test = df_test.iloc[:,-1]

attack_test = df_test.iloc[:,-11]

#min-max scaling
testdata_norm = (testdata - testdata.min()) / (testdata.max() - testdata.min())
testdata_norm = testdata_norm.fillna(0)

testdata_norm.head(10)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
87395,1.928715e-02,0.856061,0.416667,0.250,0.000948,0.000729,0.000074,0.000024,0.000015,0.996078,...,0.00,0.00,0.000000,0.062500,0.0,0.0,0.000000,0.00,0.000000,0.0
171525,6.666668e-08,0.901515,0.166667,0.375,0.000105,0.000000,0.000007,0.000000,0.250000,0.996078,...,0.28,0.26,0.288889,0.203125,0.0,0.0,0.000000,0.26,0.213115,0.0
100997,3.536851e-03,0.856061,0.000000,0.000,0.000527,0.000182,0.000076,0.000006,0.000033,0.243137,...,0.06,0.06,0.000000,0.093750,0.0,0.0,0.000000,0.08,0.098361,0.0
106304,1.500000e-07,0.909091,0.000000,0.375,0.000105,0.000000,0.000014,0.000000,0.111111,0.996078,...,0.04,0.04,0.044444,0.046875,0.0,0.0,0.000000,0.04,0.049180,0.0
170606,1.929705e-02,0.856061,0.416667,0.250,0.000948,0.000729,0.000040,0.000024,0.000015,0.996078,...,0.00,0.02,0.000000,0.000000,0.0,0.0,0.033333,0.36,0.000000,0.0
113031,1.333334e-07,0.901515,0.000000,0.375,0.000105,0.000000,0.000011,0.000000,0.125000,0.996078,...,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.02,0.016393,0.0
70458,1.333334e-07,0.022727,0.000000,0.375,0.000105,0.000000,0.000014,0.000000,0.125000,0.996078,...,0.02,0.02,0.022222,0.031250,0.0,0.0,0.000000,0.04,0.032787,0.0
138025,1.500000e-07,0.901515,0.166667,0.375,0.000105,0.000000,0.000007,0.000000,0.111111,0.996078,...,0.54,0.52,0.266667,0.406250,0.0,0.0,0.000000,0.52,0.426230,0.0
135044,1.166667e-07,0.901515,0.166667,0.375,0.000105,0.000000,0.000007,0.000000,0.142857,0.996078,...,0.30,0.30,0.333333,0.375000,0.0,0.0,0.000000,0.32,0.393443,0.0
161530,5.000001e-08,0.901515,0.166667,0.375,0.000105,0.000000,0.000007,0.000000,0.333333,0.996078,...,0.06,0.06,0.066667,0.296875,0.0,0.0,0.000000,0.06,0.311475,0.0


In [23]:
#test_X = torch.tensor(testdata_norm.values, dtype=torch.float32)
#test_Y = torch.tensor(testlabel.values, dtype=torch.long) 

#test = TensorDataset(test_X, test_Y)
#test_loader = DataLoader(test, batch_size=100)

In [24]:
# preparing 10 types of binary classification set
test_X = torch.tensor(testdata_norm.values, dtype=torch.float32)

test_analysis_Y = torch.tensor(analysis_test.values, dtype=torch.long) 
test_backdoor_Y = torch.tensor(backdoor_test.values, dtype=torch.long) 
test_dos_Y = torch.tensor(dos_test.values, dtype=torch.long) 
test_exploits_Y = torch.tensor(exploits_test.values, dtype=torch.long) 
test_fuzzers_Y = torch.tensor(fuzzers_test.values, dtype=torch.long) 
test_generic_Y = torch.tensor(generic_test.values, dtype=torch.long) 
test_normal_Y = torch.tensor(normal_test.values, dtype=torch.long) 
test_reconnaissance_Y = torch.tensor(reconnaissance_test.values, dtype=torch.long) 
test_shellcode_Y = torch.tensor(shellcode_test.values, dtype=torch.long) 
test_worms_Y = torch.tensor(worms_test.values, dtype=torch.long) 

test_analysis = TensorDataset(test_X, test_analysis_Y)
test_backdoor = TensorDataset(test_X, test_backdoor_Y)
test_dos = TensorDataset(test_X, test_dos_Y)
test_exploits = TensorDataset(test_X, test_exploits_Y)
test_fuzzers = TensorDataset(test_X, test_fuzzers_Y)
test_generic = TensorDataset(test_X, test_generic_Y)
test_normal = TensorDataset(test_X, test_normal_Y)
test_reconnaissance = TensorDataset(test_X, test_reconnaissance_Y)
test_shellcode = TensorDataset(test_X, test_shellcode_Y)
test_worms = TensorDataset(test_X, test_worms_Y)


In [25]:
test_analysis_loader = DataLoader(test_analysis, batch_size=100, shuffle=True)
test_backdoor_loader = DataLoader(test_backdoor, batch_size=100, shuffle=True)
test_dos_loader = DataLoader(test_dos, batch_size=100, shuffle=True)
test_exploits_loader = DataLoader(test_exploits, batch_size=100, shuffle=True)
test_fuzzers_loader = DataLoader(test_fuzzers, batch_size=100, shuffle=True)
test_generic_loader = DataLoader(test_generic, batch_size=100, shuffle=True)
test_normal_loader = DataLoader(test_normal, batch_size=100, shuffle=True)
test_reconnaissance_loader = DataLoader(test_reconnaissance, batch_size=100, shuffle=True)
test_shellcode_loader = DataLoader(test_shellcode, batch_size=100, shuffle=True)
test_worms_loader = DataLoader(test_worms, batch_size=100, shuffle=True)
# drop_last = True

In [26]:
with torch.no_grad():
    correct = 0
    model_analysis.eval()
    test_x, test_y = Variable(test_X), Variable(test_analysis_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_analysis = model_analysis(test_x)
    pred = torch.max(output_test_analysis.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 51929/52603 (99%)



In [27]:
twoS_test_data = pd.DataFrame()
twoS_test_data["analysis_pred"]=output_test_analysis.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

,analysis_pred
0,3.560250e-05
1,3.176794e-15
2,3.897592e-23
3,6.015347e-02
4,7.893937e-05
5,4.772996e-04
6,6.015347e-02
7,3.337896e-24
8,4.604237e-16
9,5.468918e-09


In [28]:
#Training the machine learning model for backdoor
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_backdoor.train()
    loss_list=[]
    for train_x, train_y in train_backdoor_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_backdoor.zero_grad()
        output = model_backdoor(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_backdoor.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_backdoor_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

In [29]:
with torch.no_grad():
    correct = 0
    model_backdoor.eval()
    train_x, train_y = Variable(train_X), Variable(train_backdoor_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_backdoor = model_backdoor(train_x)
    pred = torch.max(output_backdoor.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 121573/122738 (99%)



In [30]:
twoS_train_data["backdoor_pred"]=output_backdoor.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

,analysis_pred,backdoor_pred
0,1.124398e-09,8.726526e-03
1,1.652119e-13,5.269351e-15
2,0.000000e+00,0.000000e+00
3,2.162503e-31,4.333672e-23
4,0.000000e+00,0.000000e+00
5,3.420381e-14,1.953596e-07
6,7.428961e-08,3.646181e-03
7,4.485195e-02,1.697461e-06
8,2.902930e-28,1.801602e-30
9,0.000000e+00,0.000000e+00


In [31]:
with torch.no_grad():
    correct = 0
    model_backdoor.eval()
    test_x, test_y = Variable(test_X), Variable(test_backdoor_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_backdoor = model_backdoor(test_x)
    pred = torch.max(output_test_backdoor.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 52081/52603 (99%)



In [32]:
twoS_test_data["backdoor_pred"]=output_test_backdoor.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

,analysis_pred,backdoor_pred
0,3.560250e-05,7.887676e-03
1,3.176794e-15,1.262875e-08
2,3.897592e-23,1.877878e-37
3,6.015347e-02,5.086634e-02
4,7.893937e-05,1.182245e-01
5,4.772996e-04,3.248699e-03
6,6.015347e-02,5.086634e-02
7,3.337896e-24,0.000000e+00
8,4.604237e-16,1.471305e-12
9,5.468918e-09,5.644661e-09


In [33]:
#Training the machine learning model for dos
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_dos.train()
    loss_list=[]
    for train_x, train_y in train_dos_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_dos.zero_grad()
        output = model_dos(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_dos.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_dos_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

In [34]:
with torch.no_grad():
    correct = 0
    model_dos.eval()
    train_x, train_y = Variable(train_X), Variable(train_dos_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_dos = model_dos(train_x)
    pred = torch.max(output_dos.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 114173/122738 (93%)



In [35]:
twoS_train_data["dos_pred"]=output_dos.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

,analysis_pred,backdoor_pred,dos_pred
0,1.124398e-09,8.726526e-03,3.991153e-02
1,1.652119e-13,5.269351e-15,7.877272e-12
2,0.000000e+00,0.000000e+00,3.184415e-29
3,2.162503e-31,4.333672e-23,7.702916e-23
4,0.000000e+00,0.000000e+00,1.401298e-45
5,3.420381e-14,1.953596e-07,1.376389e-12
6,7.428961e-08,3.646181e-03,2.460391e-06
7,4.485195e-02,1.697461e-06,7.687041e-02
8,2.902930e-28,1.801602e-30,8.214868e-15
9,0.000000e+00,0.000000e+00,1.610710e-11


In [36]:
with torch.no_grad():
    correct = 0
    model_dos.eval()
    test_x, test_y = Variable(test_X), Variable(test_dos_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_dos = model_dos(test_x)
    pred = torch.max(output_test_dos.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 48904/52603 (93%)



In [37]:
twoS_test_data["dos_pred"]=output_test_dos.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

,analysis_pred,backdoor_pred,dos_pred
0,3.560250e-05,7.887676e-03,7.966983e-03
1,3.176794e-15,1.262875e-08,2.311947e-13
2,3.897592e-23,1.877878e-37,2.500489e-12
3,6.015347e-02,5.086634e-02,3.403750e-01
4,7.893937e-05,1.182245e-01,3.403750e-01
5,4.772996e-04,3.248699e-03,5.440044e-03
6,6.015347e-02,5.086634e-02,3.403750e-01
7,3.337896e-24,0.000000e+00,3.022529e-16
8,4.604237e-16,1.471305e-12,9.431128e-13
9,5.468918e-09,5.644661e-09,2.924885e-12


In [38]:
#Training the machine learning model for exploits
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_exploits.train()
    loss_list=[]
    for train_x, train_y in train_exploits_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_exploits.zero_grad()
        output = model_exploits(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_exploits.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_exploits_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

In [39]:
with torch.no_grad():
    correct = 0
    model_exploits.eval()
    train_x, train_y = Variable(train_X), Variable(train_exploits_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_exploits = model_exploits(train_x)
    pred = torch.max(output_exploits.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 109655/122738 (89%)



In [40]:
twoS_train_data["exploits_pred"]=output_exploits.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

,analysis_pred,backdoor_pred,dos_pred,exploits_pred
0,1.124398e-09,8.726526e-03,3.991153e-02,3.365455e-02
1,1.652119e-13,5.269351e-15,7.877272e-12,1.598304e-07
2,0.000000e+00,0.000000e+00,3.184415e-29,5.632558e-33
3,2.162503e-31,4.333672e-23,7.702916e-23,2.178998e-24
4,0.000000e+00,0.000000e+00,1.401298e-45,3.570904e-39
5,3.420381e-14,1.953596e-07,1.376389e-12,4.690864e-10
6,7.428961e-08,3.646181e-03,2.460391e-06,9.949273e-01
7,4.485195e-02,1.697461e-06,7.687041e-02,7.992063e-01
8,2.902930e-28,1.801602e-30,8.214868e-15,2.190658e-09
9,0.000000e+00,0.000000e+00,1.610710e-11,8.385333e-22


In [41]:
with torch.no_grad():
    correct = 0
    model_exploits.eval()
    test_x, test_y = Variable(test_X), Variable(test_exploits_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_exploits = model_exploits(test_x)
    pred = torch.max(output_test_exploits.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

tensor([1, 0, 0,  ..., 1, 0, 0], device='cuda:0')
correct _rate: 45506/52603 (87%)



In [42]:
twoS_test_data["exploits_pred"]=output_test_exploits.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

,analysis_pred,backdoor_pred,dos_pred,exploits_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12


In [43]:
#Training the machine learning model for fuzzers
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_fuzzers.train()
    loss_list=[]
    for train_x, train_y in train_fuzzers_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_fuzzers.zero_grad()
        output = model_fuzzers(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_fuzzers.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_fuzzers_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_fuzzers.eval()
    train_x, train_y = Variable(train_X), Variable(train_fuzzers_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_fuzzers = model_fuzzers(train_x)
    pred = torch.max(output_fuzzers.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["fuzzers_pred"]=output_fuzzers.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_fuzzers.eval()
    test_x, test_y = Variable(test_X), Variable(test_fuzzers_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_fuzzers = model_fuzzers(test_x)
    pred = torch.max(output_test_fuzzers.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["fuzzers_pred"]=output_test_fuzzers.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 115365/122738 (94%)

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 49193/52603 (94%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14


In [44]:
#Training the machine learning model for generic
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_generic.train()
    loss_list=[]
    for train_x, train_y in train_generic_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_generic.zero_grad()
        output = model_generic(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_generic.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_generic_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_generic.eval()
    train_x, train_y = Variable(train_X), Variable(train_generic_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_generic = model_generic(train_x)
    pred = torch.max(output_generic.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["generic_pred"]=output_generic.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_generic.eval()
    test_x, test_y = Variable(test_X), Variable(test_generic_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_generic = model_generic(test_x)
    pred = torch.max(output_test_generic.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["generic_pred"]=output_test_generic.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 122121/122738 (99%)

tensor([0, 1, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 52325/52603 (99%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00


In [45]:
#Training the machine learning model for normal
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_normal.train()
    loss_list=[]
    for train_x, train_y in train_normal_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_normal.zero_grad()
        output = model_normal(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_normal.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_normal_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_normal.eval()
    train_x, train_y = Variable(train_X), Variable(train_normal_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_normal = model_normal(train_x)
    pred = torch.max(output_normal.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["normal_pred"]=output_normal.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_normal.eval()
    test_x, test_y = Variable(test_X), Variable(test_normal_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_normal = model_normal(test_x)
    pred = torch.max(output_test_normal.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["normal_pred"]=output_test_normal.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 116607/122738 (95%)

tensor([0, 0, 1,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 49388/52603 (94%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03,9.596084e-05
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00,3.249956e-09
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09,9.999993e-01
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03,2.816626e-07
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01,3.566113e-02
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03,5.092975e-04
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02,1.152405e-12
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00,3.053402e-06
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00,4.305239e-12
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00,2.408436e-10


In [46]:
#Training the machine learning model for reconnaissance
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_reconnaissance.train()
    loss_list=[]
    for train_x, train_y in train_reconnaissance_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_reconnaissance.zero_grad()
        output = model_reconnaissance(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_reconnaissance.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_reconnaissance_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_reconnaissance.eval()
    train_x, train_y = Variable(train_X), Variable(train_reconnaissance_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_reconnaissance = model_reconnaissance(train_x)
    pred = torch.max(output_reconnaissance.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["reconnaissance_pred"]=output_reconnaissance.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_reconnaissance.eval()
    test_x, test_y = Variable(test_X), Variable(test_reconnaissance_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_reconnaissance = model_reconnaissance(test_x)
    pred = torch.max(output_test_reconnaissance.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["reconnaissance_pred"]=output_test_reconnaissance.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 119371/122738 (97%)

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 51077/52603 (97%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred,reconnaissance_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03,9.596084e-05,2.108540e-01
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00,3.249956e-09,6.647118e-09
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09,9.999993e-01,2.364401e-12
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03,2.816626e-07,5.511780e-02
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01,3.566113e-02,6.557396e-09
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03,5.092975e-04,9.699320e-01
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02,1.152405e-12,2.722271e-02
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00,3.053402e-06,1.455113e-23
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00,4.305239e-12,2.854396e-12
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00,2.408436e-10,7.276298e-09


In [47]:
#Training the machine learning model for shellcode
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_shellcode.train()
    loss_list=[]
    for train_x, train_y in train_shellcode_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_shellcode.zero_grad()
        output = model_shellcode(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_shellcode.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_shellcode_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_shellcode.eval()
    train_x, train_y = Variable(train_X), Variable(train_shellcode_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_shellcode = model_shellcode(train_x)
    pred = torch.max(output_shellcode.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["shellcode_pred"]=output_shellcode.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_shellcode.eval()
    test_x, test_y = Variable(test_X), Variable(test_shellcode_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_shellcode = model_shellcode(test_x)
    pred = torch.max(output_test_shellcode.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["shellcode_pred"]=output_test_shellcode.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 122037/122738 (99%)

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 52268/52603 (99%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred,reconnaissance_pred,shellcode_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03,9.596084e-05,2.108540e-01,1.211897e-10
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00,3.249956e-09,6.647118e-09,3.966620e-15
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09,9.999993e-01,2.364401e-12,1.275312e-22
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03,2.816626e-07,5.511780e-02,3.978869e-07
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01,3.566113e-02,6.557396e-09,3.080114e-28
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03,5.092975e-04,9.699320e-01,4.188878e-03
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02,1.152405e-12,2.722271e-02,7.443461e-22
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00,3.053402e-06,1.455113e-23,8.622137e-19
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00,4.305239e-12,2.854396e-12,3.015583e-17
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00,2.408436e-10,7.276298e-09,4.455965e-15


In [48]:
#Training the machine learning model for worms
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model_worms.train()
    loss_list=[]
    for train_x, train_y in train_worms_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer_worms.zero_grad()
        output = model_worms(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer_worms.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_worms_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

with torch.no_grad():
    correct = 0
    model_worms.eval()
    train_x, train_y = Variable(train_X), Variable(train_worms_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output_worms = model_worms(train_x)
    pred = torch.max(output_worms.data, 1)[1]
    #print(output.data.softmax(dim=1))
    #print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_train_data["worms_pred"]=output_worms.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_train_data.head(10)

with torch.no_grad():
    correct = 0
    model_worms.eval()
    test_x, test_y = Variable(test_X), Variable(test_worms_Y)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    output_test_worms = model_worms(test_x)
    pred = torch.max(output_test_worms.data, 1)[1]
    print(pred)
    correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

twoS_test_data["worms_pred"]=output_test_worms.data.softmax(dim=1)[:,1].to('cpu').detach().numpy().tolist()
twoS_test_data.head(10)

correct _rate: 122655/122738 (100%)

tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
correct _rate: 52566/52603 (100%)



,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred,reconnaissance_pred,shellcode_pred,worms_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03,9.596084e-05,2.108540e-01,1.211897e-10,1.767702e-02
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00,3.249956e-09,6.647118e-09,3.966620e-15,2.588737e-07
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09,9.999993e-01,2.364401e-12,1.275312e-22,1.449428e-09
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03,2.816626e-07,5.511780e-02,3.978869e-07,1.782585e-04
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01,3.566113e-02,6.557396e-09,3.080114e-28,1.767702e-02
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03,5.092975e-04,9.699320e-01,4.188878e-03,1.655293e-03
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02,1.152405e-12,2.722271e-02,7.443461e-22,3.269916e-06
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00,3.053402e-06,1.455113e-23,8.622137e-19,3.119740e-21
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00,4.305239e-12,2.854396e-12,3.015583e-17,6.671641e-14
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00,2.408436e-10,7.276298e-09,4.455965e-15,9.368664e-06


In [49]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 10)
        self.bc1 = nn.BatchNorm1d(100)
        self.bc2 = nn.BatchNorm1d(100)
        self.bc3 = nn.BatchNorm1d(100)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bc1(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc2(x)
        x = self.bc2(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc3(x)
        x = self.bc3(x)
        x = F.relu(x) # ReLU: max(x, 0)
        x = self.fc4(x)
        # return F.log_softmax(x, dim=1)
        return x

model2 = Net2()

model2 = model2.to(device)

In [50]:
twoS_train_data.head(10)

,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred,reconnaissance_pred,shellcode_pred,worms_pred
0,1.124398e-09,8.726526e-03,3.991153e-02,3.365455e-02,8.710764e-02,4.430264e-03,9.014773e-02,8.154392e-01,2.590687e-01,1.482409e-06
1,1.652119e-13,5.269351e-15,7.877272e-12,1.598304e-07,1.411883e-24,1.000000e+00,2.550684e-11,8.139295e-13,2.812653e-19,4.668656e-14
2,0.000000e+00,0.000000e+00,3.184415e-29,5.632558e-33,2.063958e-13,0.000000e+00,1.000000e+00,1.236800e-06,1.617778e-06,8.691823e-10
3,2.162503e-31,4.333672e-23,7.702916e-23,2.178998e-24,7.995093e-28,0.000000e+00,1.000000e+00,2.208516e-14,4.049888e-07,6.476430e-07
4,0.000000e+00,0.000000e+00,1.401298e-45,3.570904e-39,1.134889e-13,0.000000e+00,1.000000e+00,1.401298e-45,1.201195e-08,4.622779e-07
5,3.420381e-14,1.953596e-07,1.376389e-12,4.690864e-10,2.127114e-19,1.000000e+00,2.798049e-09,9.179554e-10,8.819387e-15,8.719113e-06
6,7.428961e-08,3.646181e-03,2.460391e-06,9.949273e-01,2.992757e-08,3.644851e-04,5.809787e-04,3.265814e-05,5.119941e-06,2.941727e-07
7,4.485195e-02,1.697461e-06,7.687041e-02,7.992063e-01,9.728054e-04,1.278100e-03,1.525132e-01,3.548868e-03,2.600758e-07,8.555398e-04
8,2.902930e-28,1.801602e-30,8.214868e-15,2.190658e-09,6.429720e-16,1.000000e+00,6.628598e-12,2.794110e-11,2.299955e-15,6.133651e-21
9,0.000000e+00,0.000000e+00,1.610710e-11,8.385333e-22,8.115931e-10,1.620969e-39,1.000000e+00,2.091365e-35,8.181914e-20,3.443747e-24


In [51]:
twoS_test_data.head(10)

,analysis_pred,backdoor_pred,dos_pred,exploits_pred,fuzzers_pred,generic_pred,normal_pred,reconnaissance_pred,shellcode_pred,worms_pred
0,3.560250e-05,7.887676e-03,7.966983e-03,7.148159e-01,7.268800e-03,5.544554e-03,9.596084e-05,2.108540e-01,1.211897e-10,1.767702e-02
1,3.176794e-15,1.262875e-08,2.311947e-13,5.005984e-11,3.865425e-19,1.000000e+00,3.249956e-09,6.647118e-09,3.966620e-15,2.588737e-07
2,3.897592e-23,1.877878e-37,2.500489e-12,5.968644e-08,6.076699e-24,5.994871e-09,9.999993e-01,2.364401e-12,1.275312e-22,1.449428e-09
3,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,6.070593e-02,6.290934e-03,2.816626e-07,5.511780e-02,3.978869e-07,1.782585e-04
4,7.893937e-05,1.182245e-01,3.403750e-01,4.163114e-01,7.005392e-04,2.421796e-01,3.566113e-02,6.557396e-09,3.080114e-28,1.767702e-02
5,4.772996e-04,3.248699e-03,5.440044e-03,8.574696e-05,1.181894e-02,4.195367e-03,5.092975e-04,9.699320e-01,4.188878e-03,1.655293e-03
6,6.015347e-02,5.086634e-02,3.403750e-01,4.163114e-01,5.579222e-02,1.471776e-02,1.152405e-12,2.722271e-02,7.443461e-22,3.269916e-06
7,3.337896e-24,0.000000e+00,3.022529e-16,2.937190e-08,9.072125e-10,1.000000e+00,3.053402e-06,1.455113e-23,8.622137e-19,3.119740e-21
8,4.604237e-16,1.471305e-12,9.431128e-13,1.007714e-09,1.271488e-16,1.000000e+00,4.305239e-12,2.854396e-12,3.015583e-17,6.671641e-14
9,5.468918e-09,5.644661e-09,2.924885e-12,1.154147e-12,8.830046e-14,1.000000e+00,2.408436e-10,7.276298e-09,4.455965e-15,9.368664e-06


In [52]:
train_X = torch.tensor(twoS_train_data.values, dtype=torch.float32)
train_Y = torch.tensor(attack_train.values, dtype=torch.long) 
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=100, shuffle=True)

In [53]:
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.03)

In [54]:
#Training the machine learning model
batch_loss_list=[]
for epoch in range(300): #learning 100 times
    #total_loss = 0
    model2.train()
    loss_list=[]
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer2.zero_grad()
        output = model2(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer2.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
    batch_loss = sum(loss_list)/len(train_loader)
    batch_loss_list.append(batch_loss.to('cpu').detach().numpy().tolist())

In [55]:
batch_loss_list

[0.48925867676734924,
 0.4670778214931488,
 0.46240898966789246,
 0.4579775631427765,
 0.4575967490673065,
 0.4556998014450073,
 0.4534989893436432,
 0.45255860686302185,
 0.45243191719055176,
 0.45057523250579834,
 0.4487278163433075,
 0.448169469833374,
 0.4482886791229248,
 0.4476920962333679,
 0.44587576389312744,
 0.44570523500442505,
 0.4458911120891571,
 0.44520244002342224,
 0.4453178942203522,
 0.4445909857749939,
 0.44441381096839905,
 0.443867027759552,
 0.4434124529361725,
 0.4430859088897705,
 0.44308799505233765,
 0.442850798368454,
 0.442493200302124,
 0.44172170758247375,
 0.4423138201236725,
 0.4416455626487732,
 0.44149744510650635,
 0.44037342071533203,
 0.4410659670829773,
 0.44115445017814636,
 0.44095513224601746,
 0.4409911036491394,
 0.4398474395275116,
 0.44018325209617615,
 0.44002532958984375,
 0.43953824043273926,
 0.4388717710971832,
 0.4399542212486267,
 0.43909549713134766,
 0.4398687183856964,
 0.43813666701316833,
 0.43912065029144287,
 0.43909817934036

In [56]:
with torch.no_grad():
    correct = 0
    model2.eval()
    train_x, train_y = Variable(train_X), Variable(train_Y)
    train_x = train_x.to(device)
    train_y = train_y.to(device)
    output = model2(train_x)
    pred = torch.max(output.data, 1)[1]
    print(pred)
    correct += pred.eq(train_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_y)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

tensor([8, 5, 6,  ..., 3, 5, 4], device='cuda:0')
correct _rate: 101267/122738 (83%)



In [57]:
test_X = torch.tensor(twoS_test_data.values, dtype=torch.float32)
test_Y = torch.tensor(attack_test.values, dtype=torch.long) 

test = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test, batch_size=1024)

In [58]:
with torch.no_grad():
    model2.eval()
    correct = 0
    pred_list = list()
    test_y_list = list()
    for test_x, test_y in test_loader:
        test_x, test_y = Variable(test_x), Variable(test_y)
        test_y_list.extend(test_y.to('cpu').detach().numpy().tolist())
        test_x = test_x.to(device)
        test_y = test_y.to(device)
        output = model2(test_x)
        pred = torch.max(output.data, 1)[1]
        pred_list.extend(pred.to('cpu').detach().numpy().tolist())
        correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_loader.dataset)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 41586/52603 (79%)



In [59]:
accuracy_score(test_y_list, pred_list)

0.7905632758587913

In [60]:
precision_score(test_y_list, pred_list, average='macro')

0.5319364526518335

In [61]:
recall_score(test_y_list, pred_list, average='macro')

0.4741092631257362

In [62]:
f1_score(test_y_list, pred_list, average='macro')

0.47478718784300983

In [63]:
cm = confusion_matrix(test_y_list, pred_list, labels=[0,1,2,3,4,5,6,7,8,9])
print(cm)

[[   48     0    73   449     4     9     3     4     7    15]
 [    0    22    15   471     3     1     2     4     2     0]
 [    6     2   150  3371    62    23    19    20    29    17]
 [   51    26   217  9110   226   266    89    66    32    39]
 [    9     3    31   676  3402    29  1092    56    32     9]
 [    1     0    12   225    16 11853     2     4     6     2]
 [   38     1   206   208  1140    97 14827    55   100    20]
 [    0    17    72   865    54    69    40  2002     5     2]
 [    0     0     5    65    66     0     9    21   171     0]
 [    1     0     2    24     2     4     1     0     0     1]]


In [ ]:
#Test RandomForest

In [64]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
clf = RandomForestClassifier(random_state=0)
clf.fit(twoS_train_data, attack_train)

RandomForestClassifier(random_state=0)

In [66]:
clf.score(twoS_train_data, attack_train)

0.911168505271391

In [67]:
#clf.fit(twoS_test_data, attack_test)

In [68]:
clf.score(twoS_test_data, attack_test)

0.7762675132597

In [69]:
clf.score(twoS_train_data, attack_train)

0.911168505271391

In [70]:
clf.fit(data_train_norm, attack_train)
clf.score(testdata_norm, attack_test)

0.7968747029637093

In [71]:
clf.score(data_train_norm, attack_train)

0.9106226270592644